<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/draft_1107.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**패키지 설치**

In [2]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
transformers 4.57.1 

In [3]:
!pip3 install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 61.7 MB/s eta 0:00:00


**upstage api**

In [4]:
# set parameter
UPSTAGE_API_KEY = "up_6QotiShLoYSOQBmusnDxusPel5Fuh"
data_path = "/content/drive/MyDrive/assignment/25-2/NLP/ewha.pdf" # folder path containing ewah.pdf and samples.csv

In [5]:
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageDocumentParseLoader
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [6]:
import re

In [7]:
!pip install openai

**ewha.pdf loading**
<br>
HTML text로 로드
<br>
기존 MD 텍스트로 로드하였으나 부칙이 일부만 로드되고 잘림. HTML은 전체가 잘 로드됨

In [8]:
loader = UpstageDocumentParseLoader(
    data_path,
    api_key=UPSTAGE_API_KEY
)

docs = loader.load()

In [9]:
clean_texts = []

for doc in docs:
    text = doc.page_content

    text = text.strip()
    clean_texts.append(text)


full_text = "\n".join(clean_texts)

In [10]:
full_text

'<h1 id=\'0\' style=\'font-size:18px\'>이화여자대학교 학칙</h1><p id=\'1\' data-category=\'paragraph\' style=\'font-size:14px\'>1946. 8. 15. 제정<br>2017. 8. 16. 개정</p><h1 id=\'2\' style=\'font-size:14px\'>제1장 총칙</h1><p id=\'3\' data-category=\'paragraph\' style=\'font-size:14px\'>제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과<br>그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에<br>공헌할 수 있는 지도여성을 양성함을 목적으로 한다.</p><br><p id=\'4\' data-category=\'paragraph\' style=\'font-size:14px\'>제2조(명칭) 본교는 이화여자대학교라 부른다.<br>제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)</p><h1 id=\'5\' style=\'font-size:14px\'>제2장 편제</h1><p id=\'6\' data-category=\'paragraph\' style=\'font-size:14px\'>제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.</p><br><p id=\'7\' data-category=\'list\' style=\'font-size:14px\'>1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범<br>대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각<br>대학”이라 한다) (개정 2016.6.16.)<br>2. 호크마(HOKMA)교양대학<br>② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대<br>학원, 디자인대학

llm에서 부칙 formatting이 제대로 진행 안 됨
<br>
따라서 부칙을 메인 조항과 분리하여 별도로 처리하도록 함

In [15]:
match = re.search(r"부칙", full_text)

if not match:
        raise ValueError("부칙 문구를 찾을 수 없습니다.")

addendum_start = match.start()

main_text = text[:addendum_start]
extra_text = text[addendum_start:]

In [16]:
print(main_text)
print(extra_text)

<h1 id='0' style='font-size:18px'>이화여자대학교 학칙</h1><p id='1' data-category='paragraph' style='font-size:14px'>1946. 8. 15. 제정<br>2017. 8. 16. 개정</p><h1 id='2' style='font-size:14px'>제1장 총칙</h1><p id='3' data-category='paragraph' style='font-size:14px'>제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과<br>그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에<br>공헌할 수 있는 지도여성을 양성함을 목적으로 한다.</p><br><p id='4' data-category='paragraph' style='font-size:14px'>제2조(명칭) 본교는 이화여자대학교라 부른다.<br>제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)</p><h1 id='5' style='font-size:14px'>제2장 편제</h1><p id='6' data-category='paragraph' style='font-size:14px'>제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.</p><br><p id='7' data-category='list' style='font-size:14px'>1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범<br>대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각<br>대학”이라 한다) (개정 2016.6.16.)<br>2. 호크마(HOKMA)교양대학<br>② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대<br>학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보<br

# MD format

## Markdown Generation

In [12]:
from openai import OpenAI

In [17]:
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)

prompt = """
You are an expert in Korean legal document analysis and Markdown formatting.

Your task:
Convert the given Ewha University regulation texts (in HTML text) into a clean Markdown document with correctly assigned heading levels.
You are given two raw texts:
1) MAIN TEXT: the main university regulations
2) EXTRA TEXT: supplementary provisions and appended regulations
Process BOTH texts together and apply the same rules.

=================================================================
MARKDOWN HEADING RULES
=================================================================

1) **Main Body Chapters (e.g., “제1장 총칙”)**:
   - Top-level header: `# [Chapter Title]`
   - Articles within chapters: `## [Article Title]`

2) **Supplementary Provisions (부칙) Section**:
   - Create a single top-level header for all supplementary provisions: `# 부칙`
   - Within this main '부칙' section, each individual supplementary provision entry
     (e.g., "부칙(1946. 8. 15 제정)", "부칙\n① 본 학칙은...")
     should be formatted as a second-level header:
     `## [Description of Supplementary Provision]`
   - Crucially, ensure *all* distinct '부칙' sections are treated as separate chapters
     and their contents as articles within them.

=================================================================
ADDITIONAL RULES
=================================================================
- Remove ALL page numbers (e.g., “2-2-1”).
- Remove HTML tags, styles, tables, or formatting artifacts from the PDF conversion.
- Preserve all meaningful text content, but organize it strictly under the specified Markdown headings.
- Do NOT hallucinate any content not present in the text.
- Do NOT output explanations or any text outside the markdown.
- Output MUST be clean markdown and ONLY markdown.

Below are the two raw texts:

---------------- MAIN TEXT START ----------------
{main_text}
---------------- MAIN TEXT END ----------------

---------------- EXTRA TEXT START ----------------
{extra_text}
---------------- EXTRA TEXT END ----------------
"""


messages = [
    {
        "role": "system",
        "content": "You are an expert in hierarchical legal information extraction and Markdown document structuring."
    },
    {
        "role": "user",
        "content": prompt.format(main_text=main_text,
                                 extra_text=extra_text,
                                 )
    }
]

md_response = client.chat.completions.create(
    model="solar-pro2",
    messages=messages
    # response_format=response_format
)

In [18]:
print(md_response.choices[0].message.content)

# 이화여자대학교 학칙

## 제1장 총칙

### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과  
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에  
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.

### 제2조(명칭)
본교는 이화여자대학교라 부른다.

### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)

## 제2장 편제

### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.  
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범  
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각  
대학”이라 한다) (개정 2016.6.16.)  
2. 호크마(HOKMA)교양대학  
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대  
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보  
건강융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한  
다). (개정 2016.6.16., 2017.5.15.)

### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8.,  
2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1)  
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집  
할 수 있다. (개정 1999.2.9., 2017.5.15.)  
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위  
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할  
전공을 결정하게 하되 이에 필요한 사항은

### MD Split
MD 택스트를 ## 장 단위로 split

In [19]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(md_response.choices[0].message.content)

In [20]:
for header in md_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")

# 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.  
### 제2조(명칭)
본교는 이화여자대학교라 부른다.  
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
{'Header 1': '이화여자대학교 학칙', 'Header 2': '제1장 총칙'}
## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건강융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한
다). (개정 2016.6.16., 2017.5.15.)  
### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8.,
2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을 이수한 후에 총장의 승인을 얻어 이수할


In [21]:
from langchain_upstage import UpstageEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
import json

## Extract Clean Markdown
불필요한 텍스트가 붙어 나와 strip 후 장 단위로 split

In [ ]:
md_content = md_response.choices[0].message.content
cleaned_md_text = md_content.replace('```markdown', '').replace('```', '').strip()

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")
    # ("###", "Header 3")
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
md_header_splits = markdown_splitter.split_text(cleaned_md_text)

for i, doc in enumerate(md_header_splits):
    print(f"--- Document {i+1} ---")
    print(f"Content: {doc.page_content[:500]}...")
    print(f"Metadata: {doc.metadata}")
    print("-----------------------")

--- Document 1 ---
Content: # 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수 연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.  
### 제2조(명칭)
본교는 이화여자대학교라 부른다.  
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)...
Metadata: {'Header 1': '이화여자대학교 학칙', 'Header 2': '제1장 총칙'}
-----------------------
--- Document 2 ---
Content: ## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).
(개정 2016.6.16., 2017.5.15.)  
[전문개정 2015.11.27.]  
### 제5조(학부 학과 전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2...
Metadata: {'Header 1': '이화여자대학교 학칙', 'Header 2': '제2장 편제'}
-----------------------
--- Document 3 ---
Co

**embedding**

In [22]:
upstage_embeddings = UpstageEmbeddings(api_key=UPSTAGE_API_KEY, model="solar-embedding-1-large")

**vector store에 저장**

In [24]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# 장 내부에 텍스트가 너무 긴 경우 (chunk size 1500 이상) 추가 split
fine_grained_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=100
)

final_documents = []
for doc in md_header_splits:
    sub_chunks_content = fine_grained_splitter.split_text(doc.page_content)
    for chunk_content in sub_chunks_content:
        # 기존 metadata 유지
        new_metadata = doc.metadata.copy()
        final_documents.append(Document(page_content=chunk_content, metadata=new_metadata))

# 부칙이 제대로 formatting 되지 않음, 비교적 단순한 구조임으로 단순 split 후 append
extra_content = fine_grained_splitter.split_text(extra_text)
for chunk_content in extra_content:
    new_metadata = {'Header 1': '이화여자대학교 학칙', 'Header 2': '부칙'}
    final_documents.append(Document(page_content=chunk_content, metadata=new_metadata))

In [25]:
# Now create the vector store from the fine_grained_documents
vectorstore = FAISS.from_documents(final_documents, upstage_embeddings)

In [26]:
for doc in final_documents[:3]:
    print(doc.page_content)
    print(doc.metadata)
    print('----------------------')

# 이화여자대학교 학칙  
## 제1장 총칙  
### 제1조(목적)
본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.  
### 제2조(명칭)
본교는 이화여자대학교라 부른다.  
### 제3조(위치)
본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)
{'Header 1': '이화여자대학교 학칙', 'Header 2': '제1장 총칙'}
----------------------
## 제2장 편제  
### 제4조(대학 및 대학원)
① 본교에는 다음 각 호의 대학을 둔다.
1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보
건강융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한
다). (개정 2016.6.16., 2017.5.15.)  
### 제5조(학부․학과․전공 및 정원)
① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표 1과 같다. (개정 2015.5.8.,
2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.1)
② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집
할 수 있다. (개정 1999.2.9., 2017.5.15.)
③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모집단위
로 입학한 학생에 대하여는 일정한 학기와 학점을

**upstage llm 불러오기**

In [27]:
llm = ChatUpstage(api_key = UPSTAGE_API_KEY, model="solar-pro2")

**prompt template**

In [28]:
prompt_template = PromptTemplate.from_template(
    """
    You are a quiz taker about Ewha Womans University's regulation.
    Please provide most correct answer for the given context in the response format below.

    Response Format:
    [Answer]: (A) answer
    [Reason]: Brief reason for your answer
    [Final Answer]: THE final answer to the question following your given reason. Answer in this format: (A) answer

    If the answer is not present in the context, please write "The information is not present in the context."

    Here is the question and context:
    ---
    Question: {question}
    ---
    Context: {context}
    """
)

**retrieval chain: 검색+생성 과정을 자동으로 처리**

In [46]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5}) #top 5 문서반환
qa = RetrievalQA.from_chain_type(llm=llm,
                                 retriever=vectorstore.as_retriever(),
                                 chain_type="stuff",
                                 chain_type_kwargs={
                                     "prompt":prompt_template,
                                     "document_variable_name": "context",
                                 },
                                 return_source_documents=True) # 원문출처받기

**쿼리 예시**

In [47]:
query = """
QUESTION) 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학점은 몇점인가?
(A) 인정 안됨
(B) 1학점
(C) 2학점
(D) 3학점
"""
# result = qa({"query": query})
result = qa.invoke(query)

In [48]:
print(result["result"])

[Answer]: (D) 3학점  
[Reason]: 제48조의2(영어 및 정보인증) 제1항에 명시된 대로, 영어 및 정보 등에 관한 일정 기준 능력 또는 자격을 취득한 경우 각각 3학점으로 인정한다고 규정되어 있습니다.  
[Final Answer]: (D) 3학점


**testset.csv load**

In [49]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/assignment/25-2/NLP/testset.csv')

prompts = data['prompts']
answers = data['answers']

ewha_prompts = prompts[:25]
ewha_answers = answers[:25]

In [50]:
print(len(prompts))
print(len(ewha_prompts))
print(ewha_prompts)

50
25
0     QUESTION1) 재학 중인 학생이 휴학을 하려면 학기 개시일로부터 며칠 이내에 ...
1     QUESTION2) '재입학은 a회에 한하여 할 수 있다. 다만 제 28조제4호에 ...
2     QUESTION3) 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는...
3     QUESTION4) 다음 보기의 학생들 중 제적을 당하지 않는 사람을 고르면?\n(...
4     QUESTION5) 2019학년도 휴먼기계바이오공학부의 입학 정원은 몇 명인가? \...
5     QUESTION6) 1980학년도 이전 입학생에 대하여 적용하는 등급에 따른 성적점...
6     QUESTION7) 사회체육학과 소속 학생에게 수여하는 학위는 무엇인가? \n(A)...
7     QUESTION8) 복수전공 신청 자격에 해당하지 않는 것은? \n(A) 1학년을 ...
8     QUESTION9) 이화여자대학교의 설립 정신은 무엇인가요?  (A) 공산주의 이념...
9     QUESTION10) 이화여자대학교의 위치는 어디인가요?  (A) 강남구  (B) ...
10    QUESTION11) 학점 기준에 따르면 1학점당 수업 시간은 몇 시간 이상이어야 ...
11    QUESTION12) 전공과 관련된 조항으로 올바른 것은?\n(A) 2 이상의 학부...
12    QUESTION13) 등록금에 관한 조항으로 올바르지 않은 것은?\n(A) 총장은 ...
13    QUESTION14) 전공과목은 어떻게 구분되는가?\n(A) 전공기초과목과 전공과목...
14    QUESTION15) 학칙개정에 관한 설명으로 옳지 않은 것은?\n(A) 학칙 개정...
15    QUESTION16)계절학기에 취득할 수 있는 최대 학점은 몇 점입니까?\n(A) ...
16    QUESTION17)졸업하기 위해 총평균 성적이 충족해야 하는 최소 기준은?\n(A...
17    QUESTION18)수업의 결석이 출석으로 인정될 수 있는 사유가

**response에서 정답만 추출**

In [51]:
import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[Final Answer\]:\s*\((A-J)\)"

    # Readjustment로 인해 dinal answer가 여러 번 등장할 수 있으므로 findall 사용
    matches = re.findall(pattern, response)

    if matches:
        # 가장 마지막 final answer만 선택
        return matches[-1].strip()
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [52]:
responses = []

for prompt in ewha_prompts:
    response = qa.invoke(prompt)
    # responses.append(response.content)
    # print(response['result'])  # debuggiing

    final_answer = extract_answer(response['result'])
    print(final_answer)
    responses.append(final_answer)

D
B
C
D
C
A
D
C
C
B
B
B
C
A
A
B
B
D
D
C
B
A
D
C
A


**print accuracy**

In [53]:
# print accuracy

cnt = 0

for answer, response in zip(answers, responses):
    print("-"*10)
    generated_answer = extract_answer(response)
    # print(response)
    # check
    if generated_answer:
        print(f"generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")


    if generated_answer == None:
        continue
    if generated_answer in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(ewha_prompts))*100}%")

----------
generated answer: D, answer: (D)
----------
generated answer: B, answer: (A)
----------
generated answer: C, answer: (C)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (B)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: C, answer: (C)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (C)
----------
generated answer: A, answer: (A)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: D, answer: (D)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (D)
----------
generated answer: B, answer: (B)
----------
generated answer: A, answer: (A)
----------
generated answer: D, 

## Majority Voting w/ 5 iterations

In [54]:
import collections

In [60]:
majority_voted_answers = []
num_iterations = 5

for idx, prompt in enumerate(ewha_prompts):
    answers_for_current_prompt = []
    for _ in range(num_iterations):
        response = qa.invoke(prompt)
        final_answer = extract_answer(response['result'])
        if final_answer:
            answers_for_current_prompt.append(final_answer)

    if answers_for_current_prompt:
        answer_counts = collections.Counter(answers_for_current_prompt)
        most_common_answer = answer_counts.most_common(1)[0][0]
        majority_voted_answers.append(most_common_answer)
        print(f"Prompt {idx+1}: Majority voted answer = {most_common_answer}")
    else:
        majority_voted_answers.append(None)
        print(f"Prompt {idx+1}: No valid answers extracted.")


Prompt 1: Majority voted answer = D
Prompt 2: Majority voted answer = B
Prompt 3: Majority voted answer = C
Prompt 4: Majority voted answer = D
Prompt 5: Majority voted answer = C
Prompt 6: Majority voted answer = A
Prompt 7: Majority voted answer = D
Prompt 8: Majority voted answer = C
Prompt 9: Majority voted answer = C
Prompt 10: Majority voted answer = B
Prompt 11: Majority voted answer = B
Prompt 12: Majority voted answer = B
Prompt 13: Majority voted answer = C
Prompt 14: Majority voted answer = A
Prompt 15: Majority voted answer = A
Prompt 16: Majority voted answer = B
Prompt 17: Majority voted answer = B
Prompt 18: Majority voted answer = D
Prompt 19: Majority voted answer = D
Prompt 20: Majority voted answer = D
Prompt 21: Majority voted answer = B
Prompt 22: Majority voted answer = A
Prompt 23: Majority voted answer = B
Prompt 24: Majority voted answer = C
Prompt 25: Majority voted answer = C


In [66]:
# print accuracy

cnt = 0

for answer, response in zip(answers, majority_voted_answers):
    print("-"*10)
    # print(response)
    # check
    if response:
        print(f"generated answer: {response}, answer: {answer}")
    else:
        print("extraction fail")


    if response == None:
        continue
    if response in answer:
        cnt += 1

print()
print(f"acc: {(cnt/len(ewha_prompts))*100}%")

----------
generated answer: D, answer: (D)
----------
generated answer: B, answer: (A)
----------
generated answer: C, answer: (C)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (B)
----------
generated answer: D, answer: (D)
----------
generated answer: C, answer: (C)
----------
generated answer: C, answer: (C)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: C, answer: (C)
----------
generated answer: A, answer: (C)
----------
generated answer: A, answer: (A)
----------
generated answer: B, answer: (B)
----------
generated answer: B, answer: (B)
----------
generated answer: D, answer: (D)
----------
generated answer: D, answer: (D)
----------
generated answer: D, answer: (D)
----------
generated answer: B, answer: (B)
----------
generated answer: A, answer: (A)
----------
generated answer: B, 